In [5]:
import xgboost as xgb
from sklearn.model_selection import cross_val_score, KFold
import pandas as pd
import numpy as np
from os import system
from numpy import inf
import pickle
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import numpy as np
from tqdm.notebook import tqdm as tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score, KFold
import pickle
from pymoo.algorithms.soo.nonconvex.pso import PSO, PSOAnimation
from pymoo.optimize import minimize
from pymoo.factory import get_termination
from pymoo.core.callback import Callback

from pymoo.core.mixed import MixedVariableGA
from pymoo.core.variable import Real, Integer
from pymoo.optimize import minimize

from pymoo.util.display.column import Column
from pymoo.util.display.output import Output

from pymoo.core.problem import ElementwiseProblem
from pymoo.core.variable import Real, Integer, Choice, Binary
import time

In [6]:
df = pd.read_csv('../preprocessing/processed_data/wine.csv')
y_wine = df[['quality']]
x_wine = df.drop(columns='quality')
x_wine = x_wine.drop(columns='Unnamed: 0')

In [7]:
ITERATIONS = 32
POPULATION = 32

In [8]:
KFOLD_SPLITS = 3

In [9]:
n_estimators_min = 3
n_estimators_max = 1000
learning_rate_min = 0.001
learning_rate_max = 0.3
max_depth_min = 3
max_depth_max = 15
subsample_min = 0.05
subsample_max = 1.0
colsample_bytree_min = 0.2
colsample_bytree_max = 1.0
gamma_min = 0
gamma_max = 10
min_child_weight_min = 0
min_child_weight_max = 10
reg_lambda_min = 0
reg_lambda_max = 1
reg_alpha_min = 0
reg_alpha_max = 1

In [10]:
class MyOutput(Output):

    def __init__(self):
        super().__init__()
        global pbar 
        pbar = tqdm(total=ITERATIONS)
        self.score = Column("score", width=13)
        self.Parameters = Column("Parameters", width=35)
        self.columns += [self.score, self.Parameters]

    def update(self, algorithm):
        super().update(algorithm)
        self.score.set(-np.min(algorithm.pop.get("F")))
        #self.Parameters.set(algorithm.pop.get("X")[0])
        pbar.update(1)
        if pbar.n == ITERATIONS: pbar.close()

# Genetic Algorithm

In [11]:
class MixedVariableProblem(ElementwiseProblem):
    def __init__(self, **kwargs):
        #params reference 1
        xgb_params = {
        'n_estimators' : Integer(bounds=(n_estimators_min, n_estimators_max)), #high num
        'learning_rate' : Real(bounds=(learning_rate_min, learning_rate_max)),
        'max_depth' : Integer(bounds=(max_depth_min, max_depth_max)),
        'subsample' : Real(bounds=(subsample_min, subsample_max)),
        'colsample_bytree' : Real(bounds=(colsample_bytree_min, colsample_bytree_max)),
        'gamma'            : Real(bounds=(gamma_min, gamma_max)),
        'min_child_weight' : Real(bounds=(min_child_weight_min, min_child_weight_max)),
        'reg_lambda'       : Real(bounds=(reg_lambda_min, reg_lambda_max)), #l2 lambda
        'reg_alpha'        : Real(bounds=(reg_alpha_min, reg_alpha_max)), #L1 alpha
        }
        super().__init__(vars=xgb_params, n_obj=1, **kwargs)

    def _evaluate(self, X, out, *args, **kwargs):
        n_estimators = X['n_estimators']
        learning_rate = X['learning_rate']
        max_depth = X['max_depth']
        subsample = X['subsample']
        colsample_bytree = X['colsample_bytree']
        gamma = X['gamma']
        min_child_weight = X['min_child_weight']
        reg_lambda = X['reg_lambda']
        reg_alpha = X['reg_alpha']
        
        model_xgboost = xgb.XGBClassifier(
            n_estimators = n_estimators,
            learning_rate = learning_rate,
            max_depth = max_depth,
            subsample = subsample,
            colsample_bytree = colsample_bytree,
            gamma = gamma,
            min_child_weight = min_child_weight,
            reg_lambda = reg_lambda,
            reg_alpha = reg_alpha,
            n_jobs = -1
            )
        
        kfold = KFold(n_splits = KFOLD_SPLITS, shuffle = True)
        
        scores = cross_val_score(model_xgboost,  x_wine, y_wine, cv = kfold, n_jobs=-1, scoring='f1_weighted')  
        result = scores.mean()     
        if result == np.nan:
            result = 0
        #print(result)
        #return result
        out["F"] = -result

In [12]:
def run_f1_ga(ITERATIONS=32, POPULATION=32):
    problem = MixedVariableProblem()

    algorithm = MixedVariableGA(pop_size=POPULATION)

    term = get_termination("n_gen", ITERATIONS)

    start_time = time.time()

    res = minimize(problem,
                   algorithm,
                   termination=term,
                   verbose=True,
                   output=MyOutput(),
                   # seed=1,
                   )

    end_time = time.time()

    execution_time = end_time - start_time
    #print(f"Execution time: {execution_time:.2f} seconds")
    #print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))
    params, score = res.X, res.F[0]
    return params, -score, execution_time


In [13]:
def full_run_ga(iterations):
    filename = './XGBoost_wine_data.csv'
    try:
        XGBoost_data = pd.read_csv(filename)
    except:
        open(filename, "a")
        XGBoost_data = pd.DataFrame(columns=['Algorithm', 'Model', 'F1', 'Time', 'params'])

    
    for i in tqdm(range(iterations)):
        params, score, execution_time = run_f1_ga()
        
        temp = pd.DataFrame({'Algorithm' : ['GA'], 
                             'Model' : ['XGBoost'], 
                            'F1' : [score], 
                            'Time' : [execution_time],
                            'params' : [params]
                            })
        
        XGBoost_data = pd.concat([XGBoost_data, temp], ignore_index=True)
        XGBoost_data[['Algorithm', 'Model', 'F1', 'Time', 'params']].to_csv(filename, index=False)

# PSO

https://pymoo.org/algorithms/soo/pso.html?highlight=pso

In [14]:
def search_range(x, max_val, min_val):
    scaled_value = min_val + (max_val - min_val) * (x / 1000)
    
    return scaled_value

In [15]:
def round_to_integers(positions, lower_bound, upper_bound):
    return np.clip(np.round(positions), lower_bound, upper_bound).astype(int)

In [25]:
import pyswarms as ps
from pyswarms.utils.functions import single_obj as fx
import numpy as np

n_estimators_min_temp, max_depth_min_temp = 0, 0
xl=np.array([
            n_estimators_min_temp, #n_estimators_min,
            learning_rate_min,
            max_depth_min_temp, #max_depth_min,
            subsample_min,
            colsample_bytree_min,
            gamma_min,
            min_child_weight_min,
            reg_lambda_min,
            reg_alpha_min,
            ])
n_estimators_max_temp, max_depth_max_temp = 1, 1
xu=np.array([
            n_estimators_max_temp, #n_estimators_max,
            learning_rate_max,
            max_depth_max_temp, #max_depth_max,
            subsample_max,
            colsample_bytree_max,
            gamma_max,
            min_child_weight_max,
            reg_lambda_max,
            reg_alpha_max,
            ])

def PSO_Optimize_F1(values):
    x = values[0] 
    model_xgboost = xgb.XGBClassifier(
        n_estimators = round_to_integers(x[0] * n_estimators_max, n_estimators_min, n_estimators_max),
        learning_rate = x[1],
        max_depth = round_to_integers(x[2] * max_depth_max, max_depth_min, max_depth_max),
        subsample = x[3],
        colsample_bytree = x[4],
        gamma = x[5],
        min_child_weight = x[6],
        reg_lambda = x[7],
        reg_alpha = x[8],
        n_jobs = -1
        )
    
    kfold = KFold(n_splits = KFOLD_SPLITS, shuffle = True)
    
    scores = cross_val_score(model_xgboost,  x_wine, y_wine, cv = kfold, n_jobs=-1, scoring='f1_weighted')  
    result = scores.mean()     

    return -result

def run_f1_pso():
    # Call an instance of PSO
    swarm_size = POPULATION
    iters = ITERATIONS
    dim = 9
    options = {'c1': 1.5, 'c2':1.5, 'w':0.5}
    constraints = (xl,xu)

    optimizer = ps.single.GlobalBestPSO(n_particles=swarm_size,
                                        dimensions=dim,
                                        options=options,
                                        ftol = -inf,
                                        bounds=constraints)
    
    start_time = time.time()

    cost, joint_vars = optimizer.optimize(objective_func = PSO_Optimize_F1, iters=iters)
    
    end_time = time.time()

    execution_time = end_time - start_time    

    params = []
    for idx in range(len(joint_vars)):
        if idx == 0:
            params.append(round_to_integers(joint_vars[idx] * n_estimators_max, n_estimators_min, n_estimators_max))
        elif idx == 2:
            params.append(round_to_integers(joint_vars[idx] * max_depth_max, max_depth_min, max_depth_max))
        else:
            params.append(joint_vars[idx])

    params_dict = {
        'n_estimators' : params[0],
        'learning_rate' : params[1],
        'max_depth' : params[2],
        'subsample' : params[3],
        'colsample_bytree' : params[4],
        'gamma' : params[5],
        'min_child_weight' : params[6],
        'reg_lambda' : params[7],
        'reg_alpha' : params[8],
                }
    return params_dict, -cost, execution_time


In [26]:
def full_run_pso(iterations):
    filename = './XGBoost_wine_data.csv'
    try:
        XGBoost_data = pd.read_csv(filename)
    except:
        open(filename, "a")
        XGBoost_data = pd.DataFrame(columns=['Algorithm', 'Model', 'F1', 'Time', 'params'])

    
    for i in tqdm(range(iterations)):
        params, score, execution_time = run_f1_pso()
        
        temp = pd.DataFrame({'Algorithm' : ['PSO'], 
                             'Model' : ['XGBoost'], 
                            'F1' : [score], 
                            'Time' : [execution_time],
                            'params' : [params]
                            })
        
        XGBoost_data = pd.concat([XGBoost_data, temp], ignore_index=True)
        XGBoost_data[['Algorithm', 'Model', 'F1', 'Time', 'params']].to_csv(filename, index=False)

# Random Search

In [17]:
from scipy.stats import uniform
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV

def random_search_int_range(min_l, max_l):
    return (np.arange(randint.ppf(0.01, min_l, max_l),randint.ppf(0.99, min_l, max_l))).astype(int)


def run_random_search_f1(iterations = 1024):
    param_distribution = {
    'n_estimators' : round_to_integers(uniform.rvs(0, 1, 1000) * n_estimators_max, n_estimators_min, n_estimators_max),
    'learning_rate' : uniform(learning_rate_min, learning_rate_max),
    'max_depth' : round_to_integers(uniform.rvs(0, 1, 1000) * max_depth_max, max_depth_min, max_depth_max),
    'subsample' : uniform(subsample_min, subsample_max),
    'colsample_bytree' : uniform(colsample_bytree_min, colsample_bytree_max),
    'gamma' : uniform(gamma_min, gamma_max),
    'min_child_weight' : random_search_int_range(min_child_weight_min, min_child_weight_max),
    'reg_alpha' : uniform(reg_alpha_min, reg_alpha_max),
    'reg_lambda' : uniform(reg_lambda_min, reg_lambda_max),
    #Not used
    #'scale_pos_weight' : random_search_int_range(scale_pos_weight_min, scale_pos_weight_max),
    #'base_score' : uniform(base_score_min, base_score_max),
    'n_jobs' :  [-1]
    }
    
    kfold = KFold(n_splits = KFOLD_SPLITS, shuffle = True)
    
    start_time = time.time()
    
    random_search_f1 = RandomizedSearchCV(xgb.XGBClassifier(), param_distribution, n_iter = iterations, n_jobs = -1, cv=kfold, verbose=False, scoring='f1_weighted')
    random_search_f1.fit(x_wine, y_wine)

    end_time = time.time()

    execution_time = end_time - start_time
    params = random_search_f1.best_params_
    score = random_search_f1.best_score_
    
    
    return params, score, execution_time

In [134]:
def full_run_rs(iterations):
    filename = './XGBoost_wine_data.csv'
    try:
        XGBoost_data = pd.read_csv(filename)
    except:
        open(filename, "a")
        XGBoost_data = pd.DataFrame(columns=['Algorithm', 'Model', 'F1', 'Time', 'params'])

    
    for i in tqdm(range(iterations)):
        params, score, execution_time = run_random_search_f1()
        
        temp = pd.DataFrame({'Algorithm' : ['RS'], 
                             'Model' : ['XGBoost'], 
                            'F1' : [score], 
                            'Time' : [execution_time],
                            'params' : [params]
                            })
        
        XGBoost_data = pd.concat([XGBoost_data, temp], ignore_index=True)
        XGBoost_data[['Algorithm', 'Model', 'F1', 'Time', 'params']].to_csv(filename, index=False)

# Optuna

In [21]:
import optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)

def objective_f1(trial):
  
      n_estimators = trial.suggest_int('n_estimators', n_estimators_min, n_estimators_max)
      learning_rate = trial.suggest_float('learning_rate', learning_rate_min, learning_rate_max)
      max_depth = trial.suggest_int('max_depth', max_depth_min, max_depth_max)
      subsample = trial.suggest_float('subsample', subsample_min, subsample_max)
      colsample_bytree = trial.suggest_float('colsample_bytree', colsample_bytree_min, colsample_bytree_max)
      gamma = trial.suggest_float('gamma', gamma_min, gamma_max)
      min_child_weight = trial.suggest_float('min_child_weight', min_child_weight_min, min_child_weight_max)
      reg_alpha = trial.suggest_float('reg_alpha', reg_alpha_min, reg_alpha_max)
      reg_lambda = trial.suggest_float('reg_lambda', reg_lambda_min, reg_lambda_max)
      
      model_xgboost = xgb.XGBClassifier(
            n_estimators = n_estimators,
            learning_rate = learning_rate,
            max_depth = max_depth,
            subsample = subsample,
            colsample_bytree = colsample_bytree,
            gamma = gamma,
            min_child_weight = min_child_weight,
            reg_lambda = reg_lambda,
            reg_alpha = reg_alpha,
            n_jobs = -1
            )

      
      kfold = KFold(n_splits = KFOLD_SPLITS, shuffle = True)
      
      result = cross_val_score(model_xgboost, x_wine, y_wine, n_jobs=-1, scoring='f1_weighted', cv=kfold).mean()
      if result == np.nan:
            result = 0

      return result

def run_optuna_f1(n_trials=1024):
    study = optuna.create_study(direction='maximize')
    
    start_time = time.time()
    study.optimize(objective_f1, n_trials=n_trials, n_jobs = -1)
    end_time = time.time()
    execution_time = end_time - start_time    
    trial = study.best_trial
    #print('F1: {}'.format(trial.value))
    #print("Best hyperparameters: {}".format(trial.params))
    return trial.params, trial.value, execution_time


In [23]:
def full_run_optuna(iterations):
    filename = './XGBoost_wine_data.csv'
    try:
        XGBoost_data = pd.read_csv(filename)
    except:
        open(filename, "a")
        XGBoost_data = pd.DataFrame(columns=['Algorithm', 'Model', 'F1', 'Time', 'params'])

    
    for i in tqdm(range(iterations)):
        params, score, execution_time = run_optuna_f1()
        
        temp = pd.DataFrame({'Algorithm' : ['Optuna'], 
                             'Model' : ['XGBoost'], 
                            'F1' : [score], 
                            'Time' : [execution_time],
                            'params' : [params]
                            })
        
        XGBoost_data = pd.concat([XGBoost_data, temp], ignore_index=True)
        XGBoost_data[['Algorithm', 'Model', 'F1', 'Time', 'params']].to_csv(filename, index=False)

# FLAML

In [29]:
from flaml import AutoML
from sklearn.model_selection import KFold, cross_val_score
import time
import numpy as np
import xgboost as xgb

def objective_f1(config, x_train, y_train, cv):
    n_estimators = int(config["n_estimators"])
    learning_rate = config["learning_rate"]
    max_depth = int(config["max_depth"])
    subsample = config["subsample"]
    colsample_bytree = config["colsample_bytree"]
    gamma = config["gamma"]
    min_child_weight = config["min_child_weight"]
    reg_alpha = config["reg_alpha"]
    reg_lambda = config["reg_lambda"]

    model_xgboost = xgb.XGBClassifier(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        gamma=gamma,
        min_child_weight=min_child_weight,
        reg_lambda=reg_lambda,
        reg_alpha=reg_alpha,
        n_jobs=-1
    )

    kfold = KFold(n_splits=cv, shuffle=True)
    
    result = cross_val_score(model_xgboost, x_train, y_train, n_jobs=-1, scoring='f1_weighted', cv=kfold).mean()
    if np.isnan(result):
        result = 0

    return result

def run_flaml_f1(x_train, y_train, n_trials=1024, cv=3):
    automl = AutoML()
    
    settings = {
        "time_budget": 600,  # set your desired time budget
        "metric": 'f1',
        "task": 'classification',
        "log_file_name": 'flaml.log',
    }
    
    automl.fit(
        x_train=x_train,
        y_train=y_train,
        task='classification',
        estimator_list=['xgboost'],
        eval_method='f1',
        cv=cv,
        **settings
    )

    best_trial = automl.best_config
    best_value = automl.best_perf
    execution_time = automl.time_elapsed

    return best_trial, best_value, execution_time

# Example usage
# best_trial, best_value, execution_time = run_flaml_f1(x_train, y_train, n_trials=1024, cv=5)
# print("Best hyperparameters:", best_trial)
# print("Best f1 score:", best_value)
# print("Execution time:", execution_time)


In [ ]:
from flaml import AutoML
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score
import xgboost as xgb

# Load the dataset
digits = load_digits()
X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=0.2, random_state=42)

# Define the XGBoost classifier class
class XGBoostClassifier(xgb.XGBClassifier):
    def fit(self, X, y, **kwargs):
        return super().fit(X, y, **kwargs)

# Define the search space for hyperparameter optimization
param_space = {
    "n_estimators": (10, 1000),
    "learning_rate": (0.01, 1.0),
    "max_depth": (3, 10),
    "subsample": (0.1, 1.0),
    "colsample_bylevel": (0.1, 1.0),
}

# Set up cross-validation (Stratified K-Fold in this example)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Define the FLAML configuration
config = {
    "task": "classification",
    "estimator": XGBoostClassifier,
    "time_budget": 60,
    "metric": "accuracy",
    "hyperparam_space": param_space,
    "cv": cv,  # Specify the cross-validation strategy
}

# Create an AutoML instance
automl = AutoML()

# Run FLAML to optimize the XGBoost classifier with hyperparameter optimization and cross-validation
automl.fit(X_train, y_train, **config)

# Make predictions on the test set
y_pred = automl.predict(X_test)

# Evaluate the performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy}")

# Print the best hyperparameters found by FLAML
best_config = automl.best_config
print("Best Hyperparameters:", best_config)
